In [1]:
# import packages
import pop_projection.Effectifs as eff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Define law of retirement
def law_ret1(age, year_emp):
    if year_emp < 2002:
        if age+1 >= 55:
            return True
        else:
            return False
    if year_emp >= 2002:
        if age+1 >= 60:
            return True
        else:
            return False

In [3]:
# Define law of reignation
def law_resignation_1(age, sexe):
    if age+1 >= 50 :
        return 0
    if sexe == 'female':
        if age+1 <= 30:
            return 0.02
        else:
            return 0.01
    if sexe == 'male':
        if age+1 <= 30:
            return 0.02
        else:
            return 0.01

In [4]:
# Define law of marriage
def law_mar1(age, sexe, typeAgent):
    """
    Return the probability of getting maried  during the following year at a given age for a given sex

    """
    if sexe == 'male':
        if typeAgent=='active':
            if age >= 25 and age <= 54:
                return 0.095
            else :
                return 0
        else:
            return 0
    
    if sexe == 'female':
        if typeAgent=='active':
            if age >= 25 and age <= 54:
                return 0.15
            else :
                return 0
        else:
            return 0


In [5]:
# Define law of replacement   
def law_replacement1(departures_, year_):
    
    '''
        assumes departures_ is a dic storing number of departures by group of the year year_
        returns a list of dics having keys : sex, age, number and group
        
    '''
    new_employees = []

    for g in departures_:
        temp = {'sex':'male', 'age' : 30, 'number':departures_[g],'group':g}
        new_employees.append(temp)
    
    return new_employees

In [6]:
# Path for input data
path ="./data/"

In [7]:
# Number of years to project
MAX_ANNEES = 100

In [8]:
# Loading data
employees = pd.read_csv(path + "employees.csv",sep=";", decimal = ",")
spouses = pd.read_csv(path + "spouses.csv",sep=";", decimal = ",")
children = pd.read_csv(path + "children.csv",sep=";", decimal = ",")

FileNotFoundError: File b'./data/employees.csv' does not exist

In [ ]:
# Display some lines in data
print("First lines of employees : ")
print(employees.head(10))
print("First lines of spouses : ")
print(spouses.head(10))
print("First lines of children : ")
print(children.head(10))

In [ ]:
# Projection of population
numbers_ = eff.simulerEffectif(employees, spouses, children, 'TV 88-90', MAX_ANNEES, law_replacement_ = law_replacement1)

# Get individual projected numbers
ind_emp_numbers = eff.individual_employees_numbers(numbers_[0])

# Get lives projected numbers
employees_proj = ind_emp_numbers[0]

# Display some lines of employees_proj
print(employees_proj.head(5))

In [ ]:
# costruct and plot Ages pyramid each 10 year, starting from year 0
for i in range(1, MAX_ANNEES//10):
    
    # group employees by age and sexe and select column of year (i-1)*10
    emp_grouped = employees_proj.groupby(['age','sex'], as_index=False)[year+str((i-1)*10)].sum()
    
    # maximum age to plot in the pyramid
    max_age = 56 # 101
    
    # minimum age to plot in the pyramid
    min_age = 19
    
    #update colum age to be age at year (i-1)*10, because age represente age at the end of simulation 
    emp_grouped['age'] = emp_grouped['age'] - MAX_ANNEES + (i-1)*10
    
    # select rows that have age between limits
    emp_grouped = emp_grouped.loc[(emp_grouped['age'] < max_age) & (emp_grouped['age'] > min_age)]
    
    # create pivot table to have numbers for each age (row) and for each sex (column)
    table = pd.pivot_table(emp_grouped, values=year+str((i-1)*10), index=['age'],  columns=['sex'], aggfunc=np.sum)
    
    # replace nas by zero
    table = table.fillna(0)
    
    #calculate percentage
    if 'male' in list(table.columns):
        table['male'] = table['male']/np.sum(table['male'])
    if 'female' in list(table.columns):
        table['female'] = table['female']/np.sum(table['female'])
    
    
    if 'female' in list(table.columns):
        table['female'] = table['female'] * (-1)
    
    # pyramids will be ploted in 4*4 grid
    plt.subplot(4 , 4, i)  # i is the index of the plot in this grid
    
    # vertical spacing between pyramids
    plt.subplots_adjust(hspace = 0.5)
    
    # min and max of x-axis (percentage)
    plt.xlim(-0.05,0.05)
    
    #title of one pyramid
    plt.title('Year ' + str((i-1)*10))
    
    # title of all pyramids
    plt.suptitle('Evolution of Ages Pyramid')
    
    #legend of all pyramids
    plt.figlegend((p_male[0], p_female[0]), ('Men', 'Women'), loc = 'lower center')
    
    # ploting the pyramid using horizontal bar (barh)
    # vertical axis --> ages, horizontal axis --> percentages
    if 'male' in list(table.columns):
        values = [0] * (max_age - min_age - 1)
        for i in range(len(table['male'])):
            values[table.index[i] - min_age - 1] = table.iloc[i]['male']
        p_male = plt.barh(list(range(min_age + 1, max_age)), values, color = (149/255,125/255,98/255) )
    
    if 'female' in list(table.columns):
        for i in range(len(table['female'])):
            values[table.index[i] - min_age - 1] = table.iloc[i]['female']
        p_female = plt.barh(list(range(min_age + 1, max_age)), values, color = (0/255,128/255,0/255))


    
    